# <span style="color:green"><center>Diplomado en Inteligencia Artificial y Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Introducción al preprocesamiento de textos</center></span>

<center> Procesamiento de Lenguaje Natural</center>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
3. [Dive into Deep Learnig](https://d2l.ai/)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Herramientas básicas de programación](#Herramientas-básicas-de-programación)
* [Lectura del conjunto de datos](#Lectura-del-conjunto-de-datos)
* [Tokenización](#Tokenización)
* [Vocabulario](#Vocabulario)
* [Poniendo todas las cosas juntas](#Poniendo-todas-las-cosas-juntas)


## <span style="color:blue">Herramientas básicas de programación</span>

Adaptadas del texto guía [Dive into Deep learning](https://d2l.ai/).

### Importa módulos

In [7]:
import collections
#from collections import defaultdict
from IPython import display
import math
from matplotlib import pyplot as plt
import os
import pandas as pd
import random
import re
#import shutil
import sys
#import tarfile
#import time
import requests
#import zipfile
import hashlib

import numpy as np
import tensorflow as tf

%matplotlib inline

# Alias
size = lambda a: tf.size(a).numpy()

## <span style="color:blue">Introducción</span>

Hemos revisado y evaluado herramientas estadísticas y desafíos de predicción.
para datos secuenciales.

Estos datos pueden adoptar muchas formas. 

El texto es uno de los ejemplos más populares de datos secuenciales.

Por ejemplo, un artículo puede verse simplemente como una secuencia de palabras o incluso como una secuencia de caracteres.

Para facilitar nuestros experimentos futuros con datos secuenciales,
dedicaremos esta sección a explicar los pasos comunes de preprocesamiento de texto.

Por lo general, estos pasos son:

1. Cargue texto como cadenas en la memoria.
2. Divida las cadenas en tokens (por ejemplo, palabras y caracteres).
3. Construya una tabla de vocabulario para mapear las claves divididas en índices numéricos.
4. Convierta texto en secuencias de índices numéricos para que los modelos puedan manipularlos fácilmente.

## <span style="color:blue">Lectura del conjunto de datos</span>

Para comenzar, cargamos texto de [*The Time Machine*] de H. G. Wells (http://www.gutenberg.org/ebooks/35).
Este es un corpus bastante pequeño de poco más de 30000 palabras, pero para el propósito de lo que queremos ilustrar, está bien.

Las colecciones de documentos más realistas contienen muchos miles de millones de palabras.
La siguiente función lee el conjunto de datos en una lista de líneas de texto, donde cada línea es una cadena.
Para simplificar, aquí ignoramos la puntuación y las mayúsculas.

### Funciones especiales de lectura

In [8]:
DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

def download(name, cache_dir=os.path.join('..', 'Datos')):
    """Download a file inserted into DATA_HUB, return the local filename."""
    assert name in DATA_HUB, f"{name} does not exist in {DATA_HUB}."
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # Hit cache
    print(f'Downloading {fname} from {url}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname


In [9]:
DATA_HUB['time_machine'] = (DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  
    """Load the time machine dataset into a list of text lines."""
    with open(download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# text lines: {len(lines)}')
print(lines[0])
print(lines[10])

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## <span style="color:blue">Tokenización</span>

La siguiente función `tokenize` 
toma una lista (`líneas`) como entrada,
donde cada lista es una secuencia de texto (por ejemplo, una línea de texto).

Cada secuencia de texto se divide en una lista de tokens.

Un *token* es la unidad básica en el texto.

Al final, se devuelve una lista de listas de tokens,
donde cada token es una cadena.

In [11]:
def tokenize(lines, token='word'): 
    """Split text lines into word or character tokens."""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## <span style="color:blue">Vocabulario</span>

El tipo de cadena del token es inconveniente para ser utilizado por modelos, que toman entradas numéricas.

Ahora construyamos un diccionario, a menudo también llamado *vocabulario*, para mapear tokens de cadena en índices numéricos únicos partir de 0.

Para hacerlo, primero contamos los tokens únicos en todos los documentos del conjunto de entrenamiento, que llamremos, un *corpus*, y luego asignamos un índice numérico a cada token único posiblemente de acuerdo con su frecuencia.

Los tokens que aparecen raramente se eliminan a menudo para reducir la complejidad.
Cualquier token que no exista en el corpus o que haya sido eliminado se asigna a un token especial desconocido. “&lt;unk&gt;”.

Opcionalmente, agregamos una lista de tokens reservados, como

+  “&lt;unk&gt;” para token eliminado o desconocido,
+ “&lt;pad&gt;” para relleno (padding),
+ “&lt;bos&gt;” para indicar el comienzo de una secuencia, y 
+ “&lt;eos&gt;” para el final de una secuencia.

### Clase Vocab

In [13]:
class Vocab:  
    """Vocabulary for text."""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = [] 
        # Sort according to frequencies
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # The index for the unknown token is 0
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens): 
    """Count token frequencies."""
    # Here `tokens` is a 1D list or 2D list
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # Flatten a list of token lists into a list of tokens
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

Construimos un vocabulario usando el conjunto de datos del texto *The Time Machine* como corpus.
Luego imprimimos los primeros tokens con sus índices.


In [14]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


Ahora podemos convertir cada línea de texto en una lista de índices numéricos.


In [15]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## <span style="color:blue">Poniendo todas las cosas juntas</span>

Usando las funciones anteriores, empaquetamos todo en la función `load_corpus_time_machine`, que devuelve `corpus`, una lista de índices de tokens, y `vocab`, el vocabulario del corpus de la máquina del tiempo.
Las modificaciones que hicimos aquí son:

1.  convertimos el texto en caracteres, no en palabras, para simplificar el entrenamiento en secciones posteriores;
2.  `corpus` es una lista única, no una lista de listas de claves, ya que cada línea de texto en el conjunto de datos de la máquina del tiempo no es necesariamente una oración o un párrafo.

In [20]:
def load_corpus_time_machine(max_tokens=-1):  
    """Return token indices and the vocabulary of the time machine dataset."""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # Since each text line in the time machine dataset is not necessarily a
    # sentence or a paragraph, flatten all the text lines into a single list
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

In [25]:
vocab.to_tokens(list(range(28)))

['<unk>',
 ' ',
 'e',
 't',
 'a',
 'i',
 'n',
 'o',
 's',
 'h',
 'r',
 'd',
 'l',
 'm',
 'u',
 'c',
 'f',
 'w',
 'g',
 'y',
 'p',
 'b',
 'v',
 'k',
 'x',
 'z',
 'j',
 'q']

In [26]:
corpus[:50]

[3,
 9,
 2,
 1,
 3,
 5,
 13,
 2,
 1,
 13,
 4,
 15,
 9,
 5,
 6,
 2,
 1,
 21,
 19,
 1,
 9,
 1,
 18,
 1,
 17,
 2,
 12,
 12,
 8,
 5,
 3,
 9,
 2,
 1,
 3,
 5,
 13,
 2,
 1,
 3,
 10,
 4,
 22,
 2,
 12,
 12,
 2,
 10,
 1,
 16]

## <span style="color:blue">Resumen</span>

* El texto es una forma importante de secuencia de datos.
* Para preprocesar texto, generalmente dividimos el texto en tokens, construimos un vocabulario para mapear cadenas de tokens en índices numéricos y convertir datos de texto en índices de tokens para que los modelos los manipulen.